Test 2-1

In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [ ]:
# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Path to the directory containing the dataset
data_dir = "/content/drive/MyDrive/MIS 548 Porject Dataset"

In [ ]:
# Data sampling and loading
def sample_data(directory, sample_size):
    files = os.listdir(directory)
    sampled_files = random.sample(files, sample_size)
    return [os.path.join(directory, file) for file in sampled_files]

def load_data(directory, sample_size):
    real_samples = sample_data(os.path.join(directory, "Real"), sample_size // 2)
    fake_samples = sample_data(os.path.join(directory, "Fake"), sample_size // 2)
    all_samples = real_samples + fake_samples
    labels = [1] * (sample_size // 2) + [0] * (sample_size // 2)
    return all_samples, labels

In [ ]:
# Load sampled data
train_samples, train_labels = load_data(os.path.join(data_dir, "Train"), 10000)
val_samples, val_labels = load_data(os.path.join(data_dir, "Validation"), 2000)
test_samples, test_labels = load_data(os.path.join(data_dir, "Test"), 2000)

In [ ]:
# Convert to DataFrame for using flow_from_dataframe
import pandas as pd
train_df = pd.DataFrame({'filename': train_samples, 'class': train_labels})
val_df = pd.DataFrame({'filename': val_samples, 'class': val_labels})
test_df = pd.DataFrame({'filename': test_samples, 'class': test_labels})

In [ ]:
# Convert class labels to strings
train_df['class'] = train_df['class'].astype(str)
val_df['class'] = val_df['class'].astype(str)
test_df['class'] = test_df['class'].astype(str)

In [ ]:
# Data preprocessing and augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]  # Add brightness adjustment
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

# Create generators
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_dataframe(
    val_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 10000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.
Found 2000 validated image filenames belonging to 2 classes.


In [ ]:
# Define the EfficientNet model architecture with regularization
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout regularization
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

16705208/16705208 [==============================] - 0s 0us/step


In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Implement Learning Rate Scheduling
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

In [ ]:
# Train the model
model.fit(train_generator,
          epochs=11,
          validation_data=val_generator,
          callbacks=[reduce_lr])

Epoch 1/11
313/313 [==============================] - 4719s 15s/step - loss: 0.2123 - accuracy: 0.9132 - val_loss: 0.5255 - val_accuracy: 0.8765 - lr: 0.0010
Epoch 2/11
313/313 [==============================] - 148s 471ms/step - loss: 0.1222 - accuracy: 0.9537 - val_loss: 0.3422 - val_accuracy: 0.9085 - lr: 0.0010
Epoch 3/11
313/313 [==============================] - 146s 466ms/step - loss: 0.1064 - accuracy: 0.9588 - val_loss: 0.1834 - val_accuracy: 0.9380 - lr: 0.0010
Epoch 4/11
313/313 [==============================] - 146s 466ms/step - loss: 0.0899 - accuracy: 0.9623 - val_loss: 0.2096 - val_accuracy: 0.9275 - lr: 0.0010
Epoch 5/11
313/313 [==============================] - 146s 467ms/step - loss: 0.0753 - accuracy: 0.9690 - val_loss: 0.1529 - val_accuracy: 0.9380 - lr: 0.0010
Epoch 6/11
313/313 [==============================] - 147s 469ms/step - loss: 0.0749 - accuracy: 0.9701 - val_loss: 0.1752 - val_accuracy: 0.9390 - lr: 0.0010
Epoch 7/11
313/313 [===========================

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

63/63 [==============================] - 701s 11s/step - loss: 0.9692 - accuracy: 0.8590
Test Loss: 0.9692012071609497
Test Accuracy: 0.859000027179718
